## mapping tests 

In [2]:
import sys,os,shutil,subprocess
import glob
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import matplotlib as mpl
import seaborn as sns
import geopandas as gpd
import random
from snipgenie import trees, app, snp_typing
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
counties = gpd.read_file('https://github.com/dmnfarrell/snipgenie/raw/master/maps/ireland_counties.zip?raw=true')

## make fake data for map/db testing

In [109]:
meta = pd.read_csv('ireland_metadata.csv',index_col=0)
meta = meta.drop(columns=['filename','pair'])
meta = meta.rename(columns={'SB':'spoligotype'})
meta=meta.set_index('name')

In [110]:
def make_points(x):
    """Make points"""

    if x.county == 'Wicklow':
        bounds=(-6.7,52.8,-6.1,53.2)
    elif x.county == 'Monaghan':
        bounds = (-6.0,54.0,-7.1,54.5)
    else:
        bounds = (-6.4,53.8,-6.1,54.2)
    points = []
    minx, miny, maxx, maxy = bounds   
    point = (random.uniform(minx, maxx), random.uniform(miny, maxy))
    return point


In [111]:
meta['coord'] = meta.apply(make_points,1)
meta[['LONG','LAT']] = pd.DataFrame(meta.coord.tolist(), index=meta.index)

In [112]:
sdist = pd.read_csv('/storage/btbgenie/all_ireland_results/snpdist.csv',index_col=0)
#print (sdist)

def find_neighbours(x, n=4):
    """find neighbours from snp distance matrix"""
  
    vals = list(x.sort_values()[1:n].index)
    return ';'.join(vals)

nearest = sdist.apply(find_neighbours,1)
#nearest
meta['nearest'] = nearest

In [113]:
#encode snp data into field
snpm = pd.read_csv('/storage/btbgenie/all_ireland_results/core.txt',sep=' ').set_index('pos').T
snpm = snpm.fillna('N')
meta['snps'] = snpm.apply(snp_typing.encode_snps,1)

In [115]:
print (meta[:4])
meta.to_csv('ireland_test_data.csv')

              sample county species spoligotype SequenceName  ClusterNumber clade                                     coord      LONG        LAT                        nearest  \
name                                                                                                                                                                              
ERR125617  ERR125617     NI     Cow      SB0140    ERR125617              7  NI-1   (-6.176099790947628, 54.08857317250175) -6.176100  54.088573  ERR125614;ERR125619;ERR125609   
ERR125622  ERR125622     NI     Cow      SB0140    ERR125622              7  NI-1   (-6.289156146752531, 53.96397871380336) -6.289156  53.963979  ERR125614;ERR125619;ERR125609   
ERR125598  ERR125598     NI     Cow      SB0140    ERR125598              7  NI-1  (-6.158677934955872, 53.970065921296445) -6.158678  53.970066  ERR125625;ERR125601;ERR125605   
ERR125612  ERR125612     NI     Cow      SB0140    ERR125612              7  NI-1  (-6.2388820348322165, 

In [ ]:
gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(pts), crs="EPSG:4326")
fig,ax=plt.subplots(1,1)
gdf.plot(ax=ax)
gdf.plot(ax=ax,color='r')